In [1]:
globals().clear
import time
import pandas as pd
import numpy as np
import seaborn as sns
sns.set()
from datetime import datetime

In [2]:
# Install Libraries

# mount Google Drive
from google.colab import drive
drive.mount('/content/colabIntern')

%cd "/content/colabIntern/MyDrive/Capstone/Capstone Project/Collab"
!ls | wc -l

Mounted at /content/colabIntern
/content/colabIntern/.shortcut-targets-by-id/1_cBH5e_TBWyCYfxXIK1yDOdiYjVxi7ae/Capstone Project/Collab
6


In [3]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.preprocessing import MinMaxScaler

import tensorflow       as tf
import keras.optimizers as op
from keras.callbacks import EarlyStopping

from tensorflow                     import keras
from tensorflow.keras.models        import Sequential
from tensorflow.keras.layers        import Dense
from tensorflow.keras.layers        import Dropout
from tensorflow.keras.optimizers    import Adam

In [4]:
!pip install tensorflow

In [119]:
# Load Dataset
df = pd.read_csv('df.csv')

In [120]:
# list of tickers for stocks in our data set. Sort the ticker list since our df will be alphabetically arranged.
tickers = ["AAPL", 'XOM', 'IBM', 'KO', 'CVX', 'BA', 'PFE', 'MSFT', 'T', 'WMT',
       'F', 'NFLX', 'JPM', 'MCD', 'GE', 'NVDA', 'JNJ', 'BAC', 'C', 'AMZN',
       'INTC', 'CSCO', 'TSLA', 'GOOGL', 'AMD', 'BABA', 'VZ', 'DIS',
       'META']
tickers.sort()

In [121]:
# For this specific trial drop PLTR since data is not complete
df = df[df.stock_ID != "PLTR"]

In [122]:
# without demographics
df = df.iloc[::,:74]

In [123]:
df['DATETIME']= pd.to_datetime(df['DATETIME'], format='%m/%d/%Y %H:%M')

In [124]:
# Sort by time so that the first 29 rows occupy the first time value for all the stocks.
# Fill in 0's for missing values for now.
df = df.set_index('DATETIME')
df = df.fillna(0)

In [125]:
# Select the length of the df ; For this file we need 16 months
# We use the first 12 months to make the first prediction, then shift window
# Then we will repeat this for the next 4 months (Hence total is 16 months needed)
df = df.sort_index().loc['2021-09-01':'2022-12-30']

In [126]:
#Set the DATETIME for fecha
DATETIME = df.index.values

In [127]:
# Sort dataframe such that it is both in sequential order, and also in alphabetical order for each day
#(i.e first entry for each time entry should be AAPL, and last should be XOM).
df.sort_values(["DATETIME", "stock_ID"], inplace=True)

In [128]:
num_companies = 29  # 29 companies in our dataset.
days = 5
step_rows  = 24 * num_companies * days  # 24 time periods per day per stock
total_rows = len(df['2021-09-01':'2022-12-30'])  # Define total length to predict on
train_rows = len(df['2021-09-01':'2022-08-31'])  # Define length of training window


scale_X = MinMaxScaler()
model = Sequential()
model.add(Dense(100, activation = 'relu', input_dim = df.iloc[:,1:-1].shape[1]))  # initiating with 100 neurons
model.add(Dense(100, activation = 'relu', input_dim = df.iloc[:,1:-1].shape[1]))
model.add(Dropout(0.25))                                                          # adding dropout to avoid overfitting
model.add(Dense(1))                                                               # output layer

opt = Adam(amsgrad = True, lr = 0.001, beta_1 = 0.79, beta_2 = 0.999)             # using Adam optimizer, at a learning rate of 0.001
model.compile(loss = 'mse', optimizer = opt)                                      # compiling model
#es = EarlyStopping(monitor='loss', mode='min', verbose=1, patience=4)

result = pd.DataFrame(columns=['DATETIME', 'ACTUAL', 'PREDICTED', 'DIFFERENCE', 'TRAIN_DURATION'])

for i in range(0, total_rows - train_rows, step_rows):
    st = time.time()
    # 1. Obtain X and y
    train  = df.iloc[i:train_rows + i, 1:]
    test   = df.iloc[train_rows + i:train_rows + i + step_rows, 1:]
    X_train, y_train = train.iloc[:, 1:], train.iloc[:, 0]
    X_test, y_test = test.iloc[:, 1:], test.iloc[:, 0]



    # 2. Scale X
    X_train = scale_X.fit_transform(X_train)
    X_test = scale_X.transform(X_test)

    # 3. Fit and Predict
    model.fit(X_train, y_train)#, epochs=20, batch_size=256)#, callbacks=[es])
    y_hat = model.predict(X_test, verbose=False)
    et = time.time()

    # 4. Save data with prediction
    fecha = DATETIME[train_rows + i:train_rows + i + step_rows]
    datos = {
        'DATETIME': fecha.ravel(),
        'ACTUAL': y_test.ravel(),
        'PREDICTED': y_hat.ravel(),
        'DIFFERENCE': abs(y_hat.ravel() - y_test.ravel()),
        'TRAIN_DURATION': np.full(y_hat.ravel().shape[0], et - st)
    }
    data = pd.DataFrame(data=datos)
    result = pd.concat([result, data], ignore_index=True)

    print("Count Down:", int((total_rows - train_rows - i) / step_rows))
    # print(data.head())

5351/5351 [==============================] - 14s 3ms/step - loss: 9718.5156
Count Down: 16
5351/5351 [==============================] - 13s 2ms/step - loss: 9299.3799
Count Down: 15
5351/5351 [==============================] - 13s 2ms/step - loss: 9128.9219
Count Down: 14
5351/5351 [==============================] - 13s 2ms/step - loss: 8961.0537
Count Down: 13
5351/5351 [==============================] - 14s 3ms/step - loss: 8783.2227
Count Down: 12
5351/5351 [==============================] - 14s 3ms/step - loss: 8591.4316
Count Down: 11
5351/5351 [==============================] - 14s 3ms/step - loss: 8364.7822
Count Down: 10
5351/5351 [==============================] - 14s 3ms/step - loss: 8150.6138
Count Down: 9
5351/5351 [==============================] - 15s 3ms/step - loss: 7916.6367
Count Down: 8
5351/5351 [==============================] - 14s 3ms/step - loss: 7634.7603
Count Down: 7
5351/5351 [==============================] - 13s 2ms/step - loss: 7392.1680
Count Down: 6
535

In [129]:
result

,DATETIME,ACTUAL,PREDICTED,DIFFERENCE,TRAIN_DURATION
0,2022-09-01 09:30:00,157.00,118.982971,38.017029,15.131263
1,2022-09-01 09:30:00,81.68,131.065720,49.385720,15.131263
2,2022-09-01 09:30:00,126.95,128.972412,2.022412,15.131263
3,2022-09-01 09:30:00,155.58,132.557983,23.022017,15.131263
4,2022-09-01 09:30:00,94.38,132.547272,38.167272,15.131263
...,...,...,...,...,...
56371,2022-12-30 15:15:00,18.41,29.702263,11.292263,13.115919
56372,2022-12-30 15:15:00,122.36,106.201744,16.158256,13.115919
56373,2022-12-30 15:15:00,39.40,28.673412,10.726588,13.115919
56374,2022-12-30 15:15:00,141.31,125.770378,15.539622,13.115919


In [130]:
metrics_df = pd.DataFrame(columns = ['stock_ID', "RMSE", "MAPE", "MPE", "MTT"])

In [131]:
metrics_df

,stock_ID,RMSE,MAPE,MPE,MTT


In [132]:
def mean_positive_error(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    error = np.mean(np.maximum((y_pred - y_true),0))
    return error

In [133]:
predicted, actual,duration=result['PREDICTED'],result['ACTUAL'],result['TRAIN_DURATION']

In [134]:
rmse = np.sqrt(mean_squared_error(result['ACTUAL'], result['PREDICTED']))
# Calculate MAPE %
mape = mean_absolute_percentage_error(result['ACTUAL'], result['PREDICTED']) * 100
# Calculate MPE %
mpe = mean_positive_error(result['ACTUAL'], result['PREDICTED'])
# Calculate MTT in seconds
mtt =duration.sum()

In [135]:
new_rows = []
for i in range(len(tickers)):
    stock_result = result[i::29]
    mtt = np.mean(stock_result['TRAIN_DURATION'])
    new_row = pd.Series([tickers[i], mtt], index=['	stock_ID','MTT'])
    new_rows.append(new_row)
metrics_df = pd.DataFrame(new_rows)

In [136]:
metrics_df

,\tstock_ID,MTT
0,AAPL,17.04303
1,AMD,17.04303
2,AMZN,17.04303
3,BA,17.04303
4,BABA,17.04303
5,BAC,17.04303
6,C,17.04303
7,CSCO,17.04303
8,CVX,17.04303
9,DIS,17.04303


In [137]:
rmse,mape,mpe,metrics_df['MTT'].sum()

(68.60549687094961, 93.2472141008941, 27.559250183570327, 494.24786504992727)

In [107]:
new_rows = []
for i in range(len(tickers)):
    stock_result = result[i::29]
    # Calculate RMSE
    rmse = np.sqrt(mean_squared_error(stock_result['ACTUAL'], stock_result['PREDICTED']))
    # Calculate MAPE %
    mape = mean_absolute_percentage_error(stock_result['ACTUAL'], stock_result['PREDICTED']) * 100
    # Calculate MPE %
    mpe = mean_positive_error(stock_result['ACTUAL'],stock_result['PREDICTED'])
    # Calculate MTT in seconds
    mtt = np.mean(stock_result['TRAIN_DURATION'])
    new_row = pd.Series([tickers[i], mape, mpe, mtt], index=['stock_ID',	'MAPE',	'MPE',	'MTT'])
    new_rows.append(new_row)
metrics_df = pd.DataFrame(new_rows)

In [108]:
metrics_df

,stock_ID,MAPE,MPE,MTT
0,AAPL,7.988830,9.452335,17.22848
1,AMD,11.090851,6.565759,17.22848
2,AMZN,8.916179,7.749815,17.22848
3,BA,7.421140,2.443563,17.22848
4,BABA,17.750618,11.958340,17.22848
5,BAC,35.691742,10.380632,17.22848
6,C,9.380543,2.350334,17.22848
7,CSCO,22.689569,8.475371,17.22848
8,CVX,8.044147,0.046292,17.22848
9,DIS,10.513157,6.042964,17.22848


In [109]:
pd.DataFrame(metrics_df.drop(['stock_ID','MTT'],axis=1).mean()).T

,MAPE,MPE
0,13.303677,6.077507
